In [1]:
import requests

In [3]:
url_pelis = f"https://beta.adalab.es/resources/apis/pelis/pelis.json"

In [4]:
peticion = requests.get(url_pelis)

In [5]:
peticion.status_code

200

In [6]:
datos_peliculas = peticion.json()

In [ ]:
datos_peliculas

In [8]:
info_peliculas = []

for pelicula in datos_peliculas:
    info_peliculas.append({"titulo": pelicula.get("titulo"), "año": pelicula.get("año"), "duracion": pelicula.get("duracion"), "genero": pelicula.get("genero"), "adultos": pelicula.get("adultos")})

In [ ]:
info_peliculas

In [10]:
import mysql.connector
from mysql.connector import errorcode

In [26]:
import mysql.connector
from mysql.connector import errorcode

cnx = None
cur = None

try:
    # 1) Conectar SIN database porque aún puede no existir
    cnx = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="AlumnaAdalab",
        # auth_plugin="mysql_native_password"  # úsalo solo si te falla la auth
    )
    print("Conectada a MySQL.")

    # 2) Crear cursor
    cur = cnx.cursor()

    # 3) Crear y seleccionar la BD
    cur.execute("CREATE DATABASE IF NOT EXISTS peliculas_bd")
    cur.execute("USE peliculas_bd")

    cnx.commit()
    print("peliculas_bd lista y seleccionada.")

except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Usuario o contraseña incorrectos.")
    else:
        print("Error MySQL:", err)
        print("Código:", getattr(err, "errno", None))
        print("SQLSTATE:", getattr(err, "sqlstate", None))

finally:
    if cur is not None:
        cur.close()
    if cnx is not None and cnx.is_connected():
        cnx.close()
        print("Conexión cerrada.")


Conectada a MySQL.
peliculas_bd lista y seleccionada.
Conexión cerrada.


In [27]:
import mysql.connector

cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="AlumnaAdalab"
)

print("Conectada:", cnx.is_connected())

mycursor = cnx.cursor()
print("Cursor creado correctamente")
mycursor.close()
cnx.close()

Conectada: True
Cursor creado correctamente


In [28]:

cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="AlumnaAdalab",
    database="peliculas_bd"
)

cursor = cnx.cursor()

print("Conexión abierta:", cnx.is_connected())


Conexión abierta: True


In [29]:
import mysql.connector

cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="AlumnaAdalab",
    database="peliculas_bd"
)

cursor = cnx.cursor()

# Asegura que la tabla exista
cursor.execute("""
CREATE TABLE IF NOT EXISTS peliculas (
    id INT AUTO_INCREMENT PRIMARY KEY,
    titulo VARCHAR(50) NOT NULL,
    anio YEAR NOT NULL,
    duracion INT NOT NULL,
    genero VARCHAR(15),
    adultos BOOLEAN
)
""")
cnx.commit()

# Insert
sql = """
INSERT INTO peliculas (titulo, anio, duracion, genero, adultos)
VALUES (%s, %s, %s, %s, %s)
"""

valores = []
for pelicula in info_peliculas:
    valores.append((
        pelicula["titulo"],
        pelicula["año"],
        pelicula["duracion"],
        pelicula["genero"],
        pelicula["adultos"]
    ))

cursor.executemany(sql, valores)
cnx.commit()

print("Películas insertadas")

# (opcional) verifica cuántas hay
cursor.execute("SELECT COUNT(*) FROM peliculas")
print("Total filas:", cursor.fetchone()[0])


Películas insertadas
Total filas: 100


In [30]:
#1. ¿Cuántas películas tienen una duración superior a 120 minutos?

cursor.execute("""
SELECT COUNT(*)
FROM peliculas
WHERE duracion > 120
""")

cantidad = cursor.fetchone()[0]
print(cantidad)


59


In [16]:
print(cnx.is_connected())


True


In [17]:
#2. ¿Cuántas películas incluyen subtítulos en español?

In [31]:
#3. ¿Cuántas películas tienen contenido adulto?

cursor.execute("""
SELECT COUNT(*)
FROM peliculas
WHERE adultos = 1
""")

cantidad = cursor.fetchone()[0]
print(cantidad)

47


In [60]:
# Cierra lo que haya (si existe)
try:
    cursor.close()
except:
    pass

try:
    cnx.close()
except:
    pass

# Reconecta
cnx = mysql.connector.connect(
    host="localhost",
    user="root",
    password="AlumnaAdalab",
    database="peliculas_bd"
)

# Cursor buffered evita muchos "out of sync"
cursor = cnx.cursor(buffered=True)

print("Reconectada:", cnx.is_connected())

Reconectada: True


In [67]:
#4. ¿Cuál es la película más antigua registrada en la base de datos?

cursor.execute("""SELECT titulo
FROM peliculas
ORDER BY anio ASC
LIMIT 1;
""")
resultado = cursor.fetchone()[0]
print(resultado)


Citizen Kane


In [85]:
try:
    cursor.fetchall()
except:
    pass


In [ ]:
#5. Muestra el promedio de duración de las películas agrupado por género.

cursor.execute("""
SELECT genero, ROUND(AVG(duracion))
FROM peliculas
GROUP BY genero
""")
resultados = cursor.fetchall()
for genero, promedio in resultados:
    print(genero, promedio)


In [ ]:
#6. ¿Cuántas películas por año se han registrado en la base de datos? Ordena de mayor a menor.
cursor.execute("""
SELECT anio, COUNT(titulo)
FROM peliculas
GROUP BY anio
ORDER BY anio DESC
""")

resultados = cursor.fetchall()

for anio, cantidad in resultados:
    print(f"{anio}: {cantidad}")


In [72]:
#7. ¿Cuál es el año con más películas en la base de datos
cursor.execute("""
SELECT anio, COUNT(*) AS total_peliculas
FROM peliculas
GROUP BY anio
ORDER BY total_peliculas DESC
LIMIT 1
""")

anio, total = cursor.fetchone()
print(f" {anio} {total} ")



 2001 5 


In [ ]:
#8. Obtén un listado de todos los géneros y cuántas películas corresponden a cada uno.

cursor.execute("""
SELECT genero, COUNT(titulo) AS total
FROM peliculas
GROUP BY genero
""")

resultados = cursor.fetchall()

for genero, total in resultados:
    print(f"{genero}: {total}")



In [ ]:
#9. Muestra todas las películas cuyo título contenga la palabra "Godfather" (puedes usar cualquier palabra).

sql = """
SELECT titulo
FROM peliculas
WHERE titulo LIKE %s
"""

valor = "%Godfather%"

cursor.execute(sql, (valor,))
resultados = cursor.fetchall()

for fila in resultados:
    print(fila[0])

The Godfather
The Godfather Part II
